#Numerical Analysis' project

Movie recommendation system

In [56]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.stats import pearsonr
from numpy.linalg import matrix_rank
import seaborn as sns
import time
import jax.numpy as jnp
import jax

Load the dataset using pandas

In [57]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [58]:
#ratings # 105339 users' ratings , 668 different users
#movies # 10329 movies

rows = np.array(ratings['userId'])
cols = np.array(ratings['movieId'])
vals = np.array(ratings['rating'])

n = rows.max() + 1 # Number of user
p = cols.max() + 1# Number of movies
N = len(vals) # Number of ratings

n , p , N

(669, 149533, 105339)

In [ ]:
movies.head()

In [ ]:
ratings.head()

In [ ]:
movies.info()

In [ ]:
ratings.info()

In [ ]:
movies.describe()

In [ ]:
ratings.describe()

In [ ]:
sns.distplot(ratings['rating'])

In [ ]:
sns.distplot(ratings['movieId'])

In [ ]:
sns.scatterplot(data = ratings , x = 'userId' , y = 'movieId' , hue = 'rating')

In [ ]:
ratings.corr()

Shuffle the data

In [59]:
indexes = np.arange(N)
np.random.seed(0) # for reproducibility
np.random.shuffle(indexes)
indexes
# Reordering the arrays
rows = rows[indexes]
cols = cols[indexes]
vals = vals[indexes]

Building the train set (80%) and the validation set (20%)

In [60]:
# Split data in training and testing
num_training = int(N * 0.8)

rows_train = rows[:num_training]
cols_train = cols[:num_training]
vals_train = vals[:num_training]
rows_test  = rows[num_training:]
cols_test  = cols[num_training:]
vals_test  = vals[num_training:]

In [61]:
# Find all the possible user ids and movie ids-> do that after the deletion
userIds_available = set()
movieIds_available = set()
for id in np.array(ratings['userId'] , dtype = int):
  userIds_available.add(id)

for id in np.array(movies['movieId'] , dtype = int):
  movieIds_available.add(id)

print(len(userIds_available) , len(movieIds_available))
#userIds_available

668 10329


In [62]:
# Make the 2 sets list in order to easily find the index of an id
userIds_available = list(userIds_available)
movieIds_available = list(movieIds_available) 

Building the matrix with the origina values

In [ ]:
#original_values = csr_matrix((vals_train, (rows_train, cols_train)), shape=(len(userIds_available), len(movieIds_available)))
#original_values = original_values.toarray()

Building the 'Ratings matrix'
Users on the rows and Movies on the columns

Initializing all the elements to 0 and then updating position (i,j) with the rating of movie j by user i if it's present

In [63]:
# Initialize the matrix to all 0
print("The number of userIds is:" , len(userIds_available))
print("The number of movieIds is:" , len(movieIds_available))
# +1 because the ids start from 1
ratings_matrix = np.zeros((len(userIds_available) , len(movieIds_available)))
ratings_matrix.shape

The number of userIds is: 668
The number of movieIds is: 10329


(668, 10329)

In [64]:
# Updating the matrix with the real ratings where possible
#ratings_matrix[rows_train , cols_train] = vals_train
def find_user_pos(id):
  return userIds_available.index(id)

def find_movie_pos(id):
  return movieIds_available.index(id)

for i in range(len(rows_train)):
  # Take the corresponding position in the matrix
  u_pos = find_user_pos(rows_train[i])
  m_pos = find_movie_pos(cols_train[i])
  # Set the known value
  ratings_matrix[u_pos , m_pos] = vals_train[i]

In [11]:
frame = pd.DataFrame(ratings_matrix, index = userIds_available , columns = movieIds_available)
print(frame)

     98304   1       2       3       ...  131050  98284   98290   98296 
1       0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
2       0.0     0.0     0.0     2.0  ...     0.0     0.0     0.0     0.0
3       0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
4       0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
5       0.0     4.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
..      ...     ...     ...     ...  ...     ...     ...     ...     ...
664     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
665     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
666     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
667     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
668     3.0     0.0     3.0     0.0  ...     0.0     3.0     2.0     2.5

[668 rows x 10329 columns]


Verifying the rank of the ratings_matrix is much greater than the minimum betweeen n and p:

$$
rank >> min(n,p)
$$

In [ ]:
rank = matrix_rank(ratings_matrix)

In [ ]:
rank

668

Checking if there are users that haven't watched any movie

Deleting rows corresponding to user that hasn't watched any movie

In [12]:
# There was a row deleted -> probably because there is a user repeated 
print("Initial shape: " , ratings_matrix.shape)
count = []
for user in range(ratings_matrix.shape[0]):
  # Save the row to delete
  if sum(ratings_matrix[user]) == 0: count.append(user)

ratings_matrix = np.delete(ratings_matrix , count , axis = 0)
print("Deleted %d rows" % len(count))
print("Final shape: " , ratings_matrix.shape)

Initial shape:  (668, 10329)
Deleted 0 rows
Final shape:  (668, 10329)


Building movie-genre correlation matrix M

$$
M_{i,j} = 
\begin{cases}
1 & \text{if movie i is of genre j}\\
0 & \text{otherwise}
\end{cases}
$$

In [65]:
# Put in a set all the genres available
genre_available = set()

for i in range(movies.shape[0]):
  genres = movies['genres'][i].split('|')
  for g in genres: genre_available.add(g)

# print("All genres available are: " , id_available , genre_available)

In [66]:
# Build the matrix
num_movies = len(movieIds_available)
print("Max movie id: " , max(movies['movieId']))
num_genres = len(genre_available)
correlation_matrix = np.zeros((num_movies , num_genres) , dtype = np.int8)

Max movie id:  149532


In [ ]:
correlation_matrix

In [16]:
frame = pd.DataFrame(correlation_matrix, index = movieIds_available , columns = genre_available)
print(frame)

        Mystery  IMAX  Romance  Crime  ...  Children  Western  Action  Film-Noir
98304         0     0        0      0  ...         0        0       0          0
1             0     0        0      0  ...         0        0       0          0
2             0     0        0      0  ...         0        0       0          0
3             0     0        0      0  ...         0        0       0          0
4             0     0        0      0  ...         0        0       0          0
...         ...   ...      ...    ...  ...       ...      ...     ...        ...
65514         0     0        0      0  ...         0        0       0          0
131050        0     0        0      0  ...         0        0       0          0
98284         0     0        0      0  ...         0        0       0          0
98290         0     0        0      0  ...         0        0       0          0
98296         0     0        0      0  ...         0        0       0          0

[10329 rows x 20 columns]


In [67]:
# Update the table with the correspondance
for i in range(movies.shape[0]):
  id = movies['movieId'][i]
  # Take the right position in the matrix
  id = movieIds_available.index(id)

  genres = movies['genres'][i].split('|')
  for pos , g in enumerate(genre_available):
    if g in genres:
      correlation_matrix[id , pos] = 1

In [18]:
frame = pd.DataFrame(correlation_matrix, index = movieIds_available , columns = genre_available)
print(frame)

        Mystery  IMAX  Romance  Crime  ...  Children  Western  Action  Film-Noir
98304         0     0        0      0  ...         0        0       0          0
1             0     0        0      0  ...         1        0       0          0
2             0     0        0      0  ...         1        0       0          0
3             0     0        1      0  ...         0        0       0          0
4             0     0        1      0  ...         0        0       0          0
...         ...   ...      ...    ...  ...       ...      ...     ...        ...
65514         0     0        0      0  ...         0        0       1          0
131050        0     0        0      0  ...         0        0       0          0
98284         0     0        1      0  ...         0        0       0          0
98290         0     0        0      0  ...         0        0       0          0
98296         0     0        0      1  ...         0        0       0          0

[10329 rows x 20 columns]


In [19]:
# Just to verify the correspondences
print(movies['movieId'][0])
print(movies['genres'][0].split('|'))

print(correlation_matrix[1])

# frame.to_csv()

1
['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']
[0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0]


Next step:
create a movie-movie matrix to find similiar movies: movies which covers the same genres

In [68]:
# Initializing the matrix to all zeros
movie_sim = np.zeros((num_movies , num_movies))

In [69]:
def similar_movies(movie1 , movie2):
  """
  movie1 and movie2 are rows of correlation_matrix
  """
  intersection = np.bitwise_and(movie1 , movie2)
  union = np.bitwise_or(movie1 , movie2)
  return sum(intersection) / sum(union)

similar_movies([1, 0, 0, 1, 0, 1] , [0, 0, 0, 1, 0, 1])


0.6666666666666666

In [31]:
# Probably this matrix is useless -> yes: to slow, we don't need it for every possible combination
for i in range(10):
  for j in range(i , 10):
    movie_sim[j , i] = similar_movies(correlation_matrix[i] , correlation_matrix[j])

In [23]:
frame = pd.DataFrame(movie_sim[0:10 , 0:10])
print(frame)

          0         1         2         3  ...         6    7    8    9
0  1.000000  0.000000  0.000000  0.000000  ...  0.000000  0.0  0.0  0.0
1  0.000000  1.000000  0.000000  0.000000  ...  0.000000  0.0  0.0  0.0
2  0.000000  0.600000  1.000000  0.000000  ...  0.000000  0.0  0.0  0.0
3  0.000000  0.166667  0.000000  1.000000  ...  0.000000  0.0  0.0  0.0
4  0.333333  0.142857  0.000000  0.666667  ...  0.000000  0.0  0.0  0.0
5  0.000000  0.200000  0.000000  0.500000  ...  0.000000  0.0  0.0  0.0
6  0.000000  0.000000  0.000000  0.000000  ...  1.000000  0.0  0.0  0.0
7  0.000000  0.166667  0.000000  1.000000  ...  0.000000  1.0  0.0  0.0
8  0.000000  0.400000  0.666667  0.000000  ...  0.000000  0.0  1.0  0.0
9  0.000000  0.000000  0.000000  0.000000  ...  0.333333  0.0  0.0  1.0

[10 rows x 10 columns]


Crate two dictionary:
    - index_cluster: (K)= movie_id (V)= number of its cluster
    - movie_cluster: (K)= cluster number (V)= list contains all similar movies

In [ ]:
movie_cluster = {}
threshold = 0.6
index_cluster = {}
movieIds_copy = movieIds_available.copy()

num_cluster = 0
while len(movieIds_copy)>0:
  for id_x in movieIds_copy:
    list_movies = []
    index_cluster[id_x] = num_cluster
    list_movies.append(id_x)
    for id_y in movieIds_copy:
      if id_x != id_y:
        sim = similar_movies(correlation_matrix[movieIds_available.index(id_x)], correlation_matrix[movieIds_available.index(id_y)])
        if sim>= threshold:
          index_cluster[id_y] = num_cluster
          list_movies.append(id_y)
          movieIds_copy.remove(id_y)
    movieIds_copy.remove(id_x)
    movie_cluster[num_cluster] = list_movies
    num_cluster += 1


In [ ]:
print(index_cluster)
print(movie_cluster.get(0))
print(movie_cluster.get(100))
print(movieIds_copy)    #this is empty now because i check all movies if similar
print(movieIds_available)   #varify all ids are untouched

Building a dictionary containing for each movie an array of similar movies

In [47]:
from tqdm.notebook import tqdm

movie_sim_dict = {}
threshold = 0.6

for i_id in tqdm(movieIds_available):
  sim_movies = []
  for j_id in movieIds_available:
    if i_id != j_id:
      sim = similar_movies(correlation_matrix[movieIds_available.index(i_id)] , correlation_matrix[movieIds_available.index(j_id)])
      if sim >= threshold:
        sim_movies.append(j_id)
  
  movie_sim_dict[int(movieIds_available[1])] = sim_movies


  0%|          | 0/10329 [00:00<?, ?it/s]

In [49]:
print(movie_sim_dict)

{98304: <function similar_movies at 0x7fa2ca6d95f0>, 1: <function similar_movies at 0x7fa2ca6d95f0>, 2: <function similar_movies at 0x7fa2ca6d95f0>, 3: <function similar_movies at 0x7fa2ca6d95f0>, 4: <function similar_movies at 0x7fa2ca6d95f0>, 5: <function similar_movies at 0x7fa2ca6d95f0>, 6: <function similar_movies at 0x7fa2ca6d95f0>, 7: <function similar_movies at 0x7fa2ca6d95f0>, 8: <function similar_movies at 0x7fa2ca6d95f0>, 9: <function similar_movies at 0x7fa2ca6d95f0>, 10: <function similar_movies at 0x7fa2ca6d95f0>, 11: <function similar_movies at 0x7fa2ca6d95f0>, 12: <function similar_movies at 0x7fa2ca6d95f0>, 13: <function similar_movies at 0x7fa2ca6d95f0>, 14: <function similar_movies at 0x7fa2ca6d95f0>, 15: <function similar_movies at 0x7fa2ca6d95f0>, 16: <function similar_movies at 0x7fa2ca6d95f0>, 17: <function similar_movies at 0x7fa2ca6d95f0>, 18: <function similar_movies at 0x7fa2ca6d95f0>, 19: <function similar_movies at 0x7fa2ca6d95f0>, 20: <function similar_mov

In [71]:
import json

dict = {
    int(movieIds_available[0]): [2 , 3 , 6],
    int(movieIds_available[1]): [1 , 5]
}

# Save the similar movies dictionary -> their calculation took 6h 50 min
with open('sim_movies.json' , 'w') as fp:
  json.dump(dict , fp , indent = 4)

In [72]:
# Open the saved data
with open('sim_movies.json' , 'r') as fp:
  movie_sim_dict_loaded = json.load(fp)

print(movie_sim_dict_loaded)

{'98304': [2, 3, 6], '1': [1, 5]}


The next step is to add possible ratings to each user according to similarities between movies: if 2 movie cover the same genre, it's possible that the rating will be the same in both of them.

At this step, just look at correlations between movies and not between users: to reduce the number of empty cells in "ratings_matrix"

In [ ]:
possible_ratings_matrix = ratings_matrix
# Over the threshold value 2 movies are considered similar
threshold = 0.6
num_of_predicted_value = 0

for i_user in range(ratings_matrix.shape[0]):
  for j_movie in range(ratings_matrix.shape[1]):
    # If user i_user has whatched movie j_movie
    if ratings_matrix[i_user , j_movie] != 0:
      for k_movie in range(ratings_matrix.shape[1]):
        # Check the similarities betweeen all the movie the user hasn't watched
        if k_movie != j_movie and ratings_matrix[i_user , k_movie] == 0: # and possible_ratings_matrix[i_user , k_movie] == 0:
          # Calculate the sim value
          sim_jk = similar_movies(correlation_matrix[j_movie] , correlation_matrix[k_movie])
          # If j and k are similar
          if sim_jk > threshold:
            possible_ratings_matrix[i_user , j_movie] = sim_jk * ratings_matrix[i_user , j_movie]
            num_of_predicted_value += 1
print(num_of_predicted_value)

# Singular value truncation (SVT) based recommender system

In [ ]:
n_max_iter = 100
threshold = 100.0
increment_tol = 1e-2

X_hat = ratings_matrix

for k in range(n_max_iter):
  X_old = X_hat.copy()
  U,s,VT = np.linalg.svd(X_hat, full_matrices=False)
  s[s < threshold] = 0
  X_hat = U @ np.diag(s) @ VT
  X_hat[rows_train,cols_train] = vals_train 
  increment = np.linalg.norm(X_hat - X_old)